# Naive Model Overview

## Idea

### Concept

This naive recommendation model leverages the **K-Nearest Neighbors** (**KNN**) algorithm to predict a user's ratings for anime they haven't rated yet. The fundamental idea is to find users who have rated the same anime in a way similar to the target user, and then use their ratings for other anime to make predictions.

### How it works

1. **Identify Similar Users**
    1. For a given user `U`, calculate the similarity between `U` and all other users based on their ratings for common anime.
    2. Metric: We use **Cosine Similarity** to measure similarity. This metric considers the angle between rating vectors and is scale-invariant, making it suitable for sparse rating data.
2. **Select Neighbors**
    1. For each anime the target user hasn’t rated, identify other users who have rated that anime.
    2. Among these users, select the top `K` most similar users (neighbors) based on the **Cosine Similarity** score.
3. **Predict Ratings**
    1. For each anime the target user hasn’t rated, predict its rating by calculating a weighted average of the ratings given by the `K` neighbors.
    2. The weight for each neighbor’s rating is determined by their similarity score with the target user.
    3. Rating for Anime `A` is predicted by formula:
    $$ 
    A = \frac{\sum{i = 1}^K (Similarity(U, N_i) \cdot Raiting(N_i, A))}{\sum{i = 1}^K (Similarity(U, N_i))}
    $$
    , where $N_i$ is i-th neighbor.


#### **Cosine Similarity**

##### **Definition**

Cosine Similarity is a metric used to measure the similarity between two non-zero vectors in a multi-dimensional space. It calculates the cosine of the angle between the vectors, where the result ranges from -1 (completely opposite) to 1 (completely identical).

##### **Mathematical Formula**

The cosine similarity between two vectors $A$ and $B$ is defined as:

$$
\text{Cosine Similarity}(A, B) = \frac{\sum_{i=1}^n A_i \cdot B_i}{\sqrt{\sum_{i=1}^n A_i^2} \cdot \sqrt{\sum_{i=1}^n B_i^2}}
$$

Where:
- $A_i$ and $B_i$ represent the components of vectors $A$ and $B$.
- The numerator is the dot product of the two vectors.
- The denominator is the product of the magnitudes of the vectors.

##### **Properties**

- **Range:**  
  $-1 \leq \text{Cosine Similarity}(A, B) \leq 1$  
  - **1:** The vectors are identical (point in the same direction).  
  - **0:** The vectors are orthogonal (no similarity).  
  - **-1:** The vectors are completely opposite.

- **Normalization:**  
  The metric normalizes the vectors, so it is scale-invariant. This means differences in magnitude (e.g., $A = [1, 2]$ vs $A = [10, 20]$) do not affect the result.

## Importing requiered libraries

Here we import some libraries requiered for this overview. 

In [52]:
import pandas as pd
import numpy as np

## Getting Data

We need to get data from provided databases and format it so we can easly find neighbours later. Getting rid of watched but not rated entires in the ratings table (there is no point in keeping unrated data we will just find next neighbour), as well as rows with missing data from the anime table.

In [53]:
# get data
file_path = "../data"
anime = pd.read_csv(file_path + "/anime.csv")
ratings = pd.read_csv(file_path + "/rating.csv")

# drop rows with missing values in 'genre' or 'rating' and missing IDs
anime = anime.dropna(subset=["genre", "rating"])
ratings = ratings.dropna(subset=["anime_id", "user_id"])
anime = anime.dropna(subset=["anime_id"])

# drop watched anime without rating
ratings = ratings[ratings["rating"] != -1]

We create mappings between indexes and IDs. then makig sure there are no missing indexes by dropping it. There is so much data that deleting stuff makes little difference. We still end up with a big data set at the end.

In [54]:
# create a mapping of anime_id to index for matrix creation
anime_id_to_index = {
    anime_id: idx for idx, anime_id in enumerate(anime["anime_id"].unique())
}
user_id_to_index = {
    user_id: idx for idx, user_id in enumerate(ratings["user_id"].unique())
}

# add index mappings to the ratings dataframe
ratings["anime_idx"] = ratings["anime_id"].map(anime_id_to_index)
ratings["user_idx"] = ratings["user_id"].map(user_id_to_index)

# drop missing ids again
ratings = ratings.dropna(subset=["anime_idx", "user_idx"])

# remove IDs
ratings.drop(["anime_id", "user_id"], axis=1, inplace=True)

Let's look at preprocessed data.

In [55]:
print(f"Amount of unique ratings: {ratings.size}\n\n")
print(ratings.info(), "\n\n")
print(ratings.head())

Amount of unique ratings: 19011438


<class 'pandas.core.frame.DataFrame'>
Index: 6337146 entries, 47 to 7813736
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   rating     int64  
 1   anime_idx  float64
 2   user_idx   int64  
dtypes: float64(1), int64(2)
memory usage: 193.4 MB
None 


     rating  anime_idx  user_idx
47       10     1709.0         0
81       10     1057.0         0
83       10      804.0         0
101      10      724.0         0
153      10      122.0         1


## Creating matrix

From individual ratings of users we want to create a matrix. This will be useful while looking for neighbours.

In [56]:
# remove duplicates
ratings = ratings.groupby(["user_idx", "anime_idx"], as_index=False).mean()

# create user-anime matrix
user_anime_matrix = ratings.pivot(
    index="user_idx", columns="anime_idx", values="rating"
).fillna(0)

# convert matrix to numpy array for faster computations
user_anime_array = user_anime_matrix.values

In [57]:
print(user_anime_matrix)

anime_idx  0.0      1.0      2.0      3.0      4.0      5.0      6.0      \
user_idx                                                                   
0              0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1              0.0      0.0      0.0      0.0      0.0      0.0      0.0   
2              0.0     10.0      0.0      0.0      0.0      0.0      0.0   
3              0.0      0.0      0.0      9.0      9.0      0.0      0.0   
4              0.0      0.0      0.0      0.0      0.0      0.0      9.0   
...            ...      ...      ...      ...      ...      ...      ...   
69595          0.0      0.0      0.0      0.0      0.0      0.0      0.0   
69596          0.0      0.0      0.0      0.0      0.0      0.0      0.0   
69597          0.0      0.0      0.0      0.0      0.0      0.0      0.0   
69598          0.0     10.0      0.0      9.0      0.0      0.0      0.0   
69599          0.0      0.0      0.0      0.0      0.0      0.0      0.0   

anime_idx  

## Model

Now let's see the model itself. It may not be intuitive what is to `fit` and what to `predict`. In presented model we `fit` all the data and predict for the subset of the `data` given in list of users we want to make predictions for.

We are forced to a use custom **cosine similarity**. We want to skip empty cells in our computations. The reason for that skipping is simple we don't want our solution to identify lack of record as 0 -  we don't want to consider 1 and 10 to be more similar than 0 and 10. From the other hand we want 9 and 10 to be closer than 0 and 10. To achive something like that we will add penalty for lacking predictions.

In [58]:
class KNNRecommender:

    def __init__(self, k=5, weight_factor=1.0, penalty_factor=0.1):
        """
        Initialize the KNN Recommender model.

        Parameters:
        - k: Number of neighbors to consider (default is 5).
        - weight_factor: Adjusts the influence of weights in the weighted average.
                         A value of 1.0 uses the weights directly.
                         A value > 1.0 increases the impact of highly similar neighbors.
                         A value < 1.0 reduces the influence of weights.
        """
        self.k = k
        self.weight_factor = weight_factor
        self.penalty_factor = penalty_factor
        self.user_anime_similarity = None
        self.train_matrix = None

    def _cosine_similarity_filtered(self, mat, min_common_ratings=2):
        """ 
        Custom cosine similarity calculation considering only non-zero values.
        Adds a penalty for pairs with few common ratings.

        Parameters:
        - mat: 2D numpy array where rows represent users and columns represent anime ratings.
        - min_common_ratings: Minimum number of common ratings for similarity to be considered.
        - penalty_factor: Reduces the similarity for pairs with fewer common ratings.
                        A higher value increases the penalty for low overlap.
        """
        n_users = mat.shape[0]
        similarity = np.zeros((n_users, n_users))

        for i in range(n_users):
            for j in range(i + 1, n_users):
                # Find indices where both users have rated the same anime (non-zero values)
                valid_indices = (mat[i, :] > 0) & (mat[j, :] > 0)
                num_common = np.sum(valid_indices)  # Number of common ratings

                if num_common >= min_common_ratings:
                    # Compute cosine similarity on these indices
                    user_i_ratings = mat[i, valid_indices]
                    user_j_ratings = mat[j, valid_indices]
                    numerator = np.dot(user_i_ratings, user_j_ratings)
                    denominator = np.sqrt(np.sum(user_i_ratings**2)) * np.sqrt(
                        np.sum(user_j_ratings**2)
                    )
                    if denominator > 0:
                        raw_similarity = numerator / denominator
                        # Apply penalty for low overlap
                        similarity[i, j] = similarity[j, i] = raw_similarity * (
                            1 - self.penalty_factor / (num_common + 1e-9)
                        )
        return similarity + 1

    def fit(self, train_matrix):
        """
        Fit the model using the training user-anime rating matrix.

        Parameters:
        - train_matrix: A matrix where rows represent users, columns represent anime,
                        and values are ratings. Missing ratings should be filled with 0.
        """
        self.train_matrix = np.array(train_matrix)
        self.user_anime_similarity = self._cosine_similarity_filtered(self.train_matrix)

    def predict(self, user_indices):
        """
        Predict ratings for specific users.

        Args:
            user_indices (list): List of user indices for whom to predict ratings.

        Returns:
            numpy.ndarray: A matrix with predicted ratings for the given users (dtype=float64).
        """
        if self.train_matrix is None or self.user_anime_similarity is None:
            raise ValueError("The model must be fit before calling predict.")

        # Initialize an empty predictions matrix for the specified users
        predictions = np.zeros((len(user_indices), self.train_matrix.shape[1]), dtype=np.float64)

        # Iterate over the specified user indices
        for idx, user in enumerate(user_indices):
            # Get the indices of the user's unrated anime (0 in the training matrix)
            unrated_anime_indices = np.where(self.train_matrix[user] == 0)[0]

            for anime in range(len(self.train_matrix[user])):
                if anime in unrated_anime_indices:
                    # Find K most similar users who have rated this anime
                    anime_ratings = self.train_matrix[:, anime]
                    similar_users = [
                        (other_user, self.user_anime_similarity[user, other_user])
                        for other_user in range(self.train_matrix.shape[0])
                        if anime_ratings[other_user] > 0 and other_user != user
                    ]
                    # Sort by similarity and take top K
                    top_k_users = sorted(similar_users, key=lambda x: x[1], reverse=True)[: self.k]

                    # Compute weighted average of ratings for the anime
                    if top_k_users:
                        numer = sum(
                            (sim**self.weight_factor) * self.train_matrix[other_user, anime]
                            for other_user, sim in top_k_users
                        )
                        denom = sum(sim**self.weight_factor for _, sim in top_k_users)
                        predictions[idx, anime] = numer / (denom + 1e-9)
                else:
                    predictions[idx, anime] = self.train_matrix[user][anime]

        return predictions

## Usage example

In [61]:
# Example dataset (rows = users, columns = anime, 0 = missing ratings)
train_matrix = np.array(
    [
        [10, 9, 5, 0, 0],   # User 0
        [0, 8, 6, 7, 0],    # User 1
        [9, 8, 0, 6, 5],    # User 2
        [0, 0, 7, 10, 8],   # User 3
        [8, 9, 6, 0, 0],    # User 4
    ]
)

# Initialize and train the model
model = KNNRecommender(k=2, weight_factor=1.0, penalty_factor=0.5)
model.fit(train_matrix)

# Predict for users 0 and 3
user_indices_to_predict = [0, 3]
predicted_ratings = model.predict(user_indices_to_predict)

# Display results
print("Predicted Ratings:\n", predicted_ratings)

Predicted Ratings:
 [[10.          9.          5.          6.49900487  6.09091476]
 [ 9.36365619  8.          7.         10.          8.        ]]
